In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import tensorflow as tf

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
"""
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0], 
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    except RuntimeError as e:
        print(e)
"""

data_dir = 'images/'

from tensorflow.keras.preprocessing.image import ImageDataGenerator

## ***Data Collection***

#datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.2)
datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)

BATCH_SIZE = 64
IMG_W=250
IMG_H=250

train_data = datagen.flow_from_directory(data_dir, target_size = (IMG_W,IMG_H), batch_size = BATCH_SIZE, class_mode = 'categorical',
                                        subset = 'training')

val_data = datagen.flow_from_directory(data_dir, target_size = (IMG_W,IMG_H), batch_size = BATCH_SIZE, class_mode = 'categorical',
                                        subset = 'validation')

import tensorflow_hub as hub
ResNet_V2_50 = 'https://tfhub.dev/google/imagenet/resnet_v1_50/classification/5'

Found 80800 images belonging to 101 classes.
Found 20199 images belonging to 101 classes.


In [3]:
%load_ext tensorboard
%tensorboard --logdir logs --port 6012


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## ***ResNet Model Building***

In [ ]:
model_ResNet = tf.keras.Sequential([
    hub.KerasLayer(ResNet_V2_50, trainable = False, input_shape = (250,250,3), name = 'Resnet_V2_50'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(101, activation = 'softmax', name = 'Output_layer')
])

model_ResNet.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

#define callback
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
import datetime,os

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tb = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


#1.ModelCheckpoint
filepath = os.path.join("logs", "{}_Resnet_V2_50_best_weight.hdf5".format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
mc = ModelCheckpoint(
    filepath, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True, 
    mode='auto'
)

#2.EarlyStopping
es=EarlyStopping(monitor='val_loss', patience=10, verbose=1)

#3.ReduceLROnPlateau
rp=ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1,
    mode='min',
    min_delta=0.001,
    cooldown=0,
    min_lr=0
)

model_ResNet.load_weights("./logs/20220122-015508_Resnet_V2_50_best_weight.hdf5.")
model_ResNet_history = model_ResNet.fit(train_data, epochs = 50, verbose = 1,validation_data=val_data,callbacks=[mc,rp,tb,es])

model_ResNet.save("Resnet_V2_50_final_{}.h5".format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
model_ResNet.save_weights('Resnet_V2_50_final_weight_{}.h5'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

model_ResNet.evaluate(val_data)

Epoch 1/50
1263/1263 [==============================] - ETA: 0s - loss: 1.4301 - accuracy: 0.6300
Epoch 00001: val_loss improved from inf to 1.77165, saving model to logs\20220122-022601_Resnet_V2_50_best_weight.hdf5
1263/1263 [==============================] - 266s 205ms/step - loss: 1.4301 - accuracy: 0.6300 - val_loss: 1.7717 - val_accuracy: 0.5709 - lr: 0.0010
Epoch 2/50
1263/1263 [==============================] - ETA: 0s - loss: 1.3808 - accuracy: 0.6394
Epoch 00002: val_loss improved from 1.77165 to 1.76630, saving model to logs\20220122-022601_Resnet_V2_50_best_weight.hdf5
1263/1263 [==============================] - 250s 198ms/step - loss: 1.3808 - accuracy: 0.6394 - val_loss: 1.7663 - val_accuracy: 0.5730 - lr: 0.0010
Epoch 3/50
  52/1263 [>.............................] - ETA: 3:17 - loss: 1.2185 - accuracy: 0.6758

## ***Efficientnet_B0 Model Building***

In [8]:
model_Efficientnet = tf.keras.Sequential([
    hub.KerasLayer(Efficientnet_b0, trainable = False, input_shape = (250,250,3), name = 'Resnet_V2_50'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(101, activation = 'softmax', name = 'Output_layer')
])

model_Efficientnet.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

model_Efficientnet.summary()


#define callback
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
import datetime,os

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tb = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


#1.ModelCheckpoint
filepath = os.path.join("logs", "weight.hdf5")
mc = ModelCheckpoint(
    filepath, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True, 
    mode='auto'
)

#2.EarlyStopping
#es=EarlyStopping(monitor='loss', patience=10, verbose=1)

#3.ReduceLROnPlateau
rp=ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1,
    mode='auto',
    min_delta=0.001,
    cooldown=0,
    min_lr=0
)


efficientnet_model = model_Efficientnet.fit(train_data, epochs = 100, verbose = 1,validation_data=val_data,callbacks=[mc,rp,tb])

model_Efficientnet.save("Efficientnet_b0_{}.h5".format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
model_Efficientnet.save_weights('Efficientnet_b0_weight_{}.h5'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

model_Efficientnet.evaluate(val_data)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Resnet_V2_50 (KerasLayer)   (None, 1000)              5330564   
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 Output_layer (Dense)        (None, 101)               101101    
                                                                 
Total params: 5,431,665
Trainable params: 101,101
Non-trainable params: 5,330,564
_________________________________________________________________


## ***Comparison Between Both Models***

In [ ]:
def plot_graph(history, history_1):
    loss_res = history.history['loss']
    loss_ef = history_1.history['loss']
    
    Accuracy_res = history.history['accuracy']
    Accuracy_ef = history_1.history['accuracy']

    epochs = range(len(history.history['loss']))

    plt.plot(epochs, loss_res, label = 'ResNet Loss')
    plt.plot(epochs, loss_ef, label = 'Efficientnet Loss')
    plt.title('Epochs Vs Loss')
    plt.xlabel('epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.figure()
    plt.plot(epochs, Accuracy_res, label = 'ResNet Accuracy')
    plt.plot(epochs, Accuracy_ef, label = 'Efficientnet Accuracy')
    plt.title('Epochs Vs Accuracy')
    plt.xlabel('epochs')
    plt.ylabel('Accuracy')
    plt.legend()

plot_graph(resnet_model, efficientnet_model)

## ***Efficientnet_B7 Model Building***

In [4]:
model_Efficientnet_b7 = tf.keras.Sequential([
    hub.KerasLayer(Efficientnet_b7, trainable = False, input_shape = (250,250,3), name = 'Efficientnet_b7'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(101, activation = 'softmax', name = 'Output_layer')
])

model_Efficientnet_b7.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = ['accuracy']
)

model_Efficientnet_b7.summary()


#define callback
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
import datetime,os

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tb = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)


#1.ModelCheckpoint
filepath = os.path.join("logs", "{}_best_weight.hdf5".format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
mc = ModelCheckpoint(
    filepath, 
    monitor='val_loss', 
    verbose=1, 
    save_best_only=True, 
    save_weights_only=True, 
    mode='auto'
)

#2.EarlyStopping
es=EarlyStopping(monitor='val_loss', patience=10, verbose=1)

#3.ReduceLROnPlateau
rp=ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=5,
    verbose=1,
    mode='auto',
    min_delta=0.001,
    cooldown=0,
    min_lr=0
)


efficientnet_b7_model = model_Efficientnet_b7.fit(train_data, epochs = 50, verbose = 1,validation_data=val_data,callbacks=[mc,rp,tb,es])

model_Efficientnet_b7.save("Efficientnet_b7_final_{}.h5".format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))
model_Efficientnet_b7.save_weights('Efficientnet_b7_final_weight_{}.h5'.format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S")))

model_Efficientnet_b7.evaluate(val_data)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Efficientnet_b7 (KerasLayer  (None, 1000)             66658680  
 )                                                               
                                                                 
 flatten (Flatten)           (None, 1000)              0         
                                                                 
 Output_layer (Dense)        (None, 101)               101101    
                                                                 
Total params: 66,759,781
Trainable params: 101,101
Non-trainable params: 66,658,680
_________________________________________________________________
Epoch 1/50
1263/1263 [==============================] - ETA: 0s - loss: 4.4228 - accuracy: 0.1632
Epoch 00001: val_loss improved from inf to 4.25395, saving model to logs\20220121-193727_best_weight.hdf5
1263/1263 [=====================

KeyboardInterrupt: 